In [1]:
import os
import json
import random
import numpy as np
import tensorflow as tf

2021-12-15 14:01:38.953583: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the senses to be used as inputs for training
x_train_sense = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the sense to be used as inputs for testing
x_test_sense = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [ ]:
# Name of directory for grids
directory_name = './data/big_3/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    try:
        data = json.load(f)
    except Exception:
        continue

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        if len(i['gridworld']) != 2500:
            print(file_name)
            continue
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_sense.append(i['sense'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()
    

In [ ]:
print(direct_count)

[2738, 6715, 15847, 21277]


In [ ]:
# Name of directory for grids
directory_name = './data/big_3test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_sense.append(i['sense'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [ ]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 50, 50) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 50, 50) )
train_in_sense = np.reshape( x_train_sense, (-1, 50, 50) ) / 8
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 50, 50) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 50, 50) )
test_in_sense = np.reshape( x_test_sense, (-1, 50, 50) ) / 8
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [ ]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (50,50) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [ ]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (50,50) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [ ]:
# Flatten the sense input
sense_input = tf.keras.layers.Input( shape = (50,50) )
flatten_sense = tf.keras.layers.Flatten()( sense_input )

In [ ]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [ ]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position, flatten_sense, flatten_local])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 7525), dtype=tf.float32, name=None), name='concatenate_26/concat:0', description="created by layer 'concatenate_26'")


In [ ]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [ ]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input, sense_input, local_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [ ]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals], y_test )

247	0	4	0
633	0	13	0
1421	0	105	0
1789	0	293	0


In [ ]:
# Train the model
history = model.fit( [train_in_grid, train_in_position, train_in_sense, train_in_locals], train_out, validation_data=([test_in_grid, test_in_position, test_in_sense, test_in_locals], test_out), epochs = 7 )

Epoch 1/7
1456/1456 [==============================] - 114s 78ms/step - loss: 0.2706 - accuracy: 0.9015 - val_loss: 0.1783 - val_accuracy: 0.9463
Epoch 2/7
1456/1456 [==============================] - 113s 77ms/step - loss: 0.1039 - accuracy: 0.9647 - val_loss: 0.3109 - val_accuracy: 0.9245
Epoch 3/7
1456/1456 [==============================] - 113s 78ms/step - loss: 0.0817 - accuracy: 0.9708 - val_loss: 0.1231 - val_accuracy: 0.9567
Epoch 4/7
1456/1456 [==============================] - 118s 81ms/step - loss: 0.0786 - accuracy: 0.9738 - val_loss: 0.1520 - val_accuracy: 0.9407
Epoch 5/7
1456/1456 [==============================] - 124s 85ms/step - loss: 0.0647 - accuracy: 0.9777 - val_loss: 0.0658 - val_accuracy: 0.9805
Epoch 6/7
1456/1456 [==============================] - 126s 86ms/step - loss: 0.0533 - accuracy: 0.9807 - val_loss: 0.0659 - val_accuracy: 0.9787
Epoch 7/7
1456/1456 [==============================] - 165s 114ms/step - loss: 0.0486 - accuracy: 0.9822 - val_loss: 0.0622 

In [ ]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals], y_test )

230	10	10	1
13	611	8	14
6	5	1506	9
9	2	1	2070


In [ ]:
model.save('./models/agent3_NN')

INFO:tensorflow:Assets written to: ./models/agent3_NN_more/assets
